[Advent of Code - Day 14](https://adventofcode.com/2019/day/14)

# Import input

In [1]:
import sys
from collections import defaultdict
from math import ceil, log10

sys.path.insert(0, "../")
from utils import aoc_input as inp

in_ = inp.download_input(year="2019", day="14")

In [2]:
in_[:5]

['9 GJSNW, 9 GHJHK => 6 DRKW',
 '1 SJNKX, 1 GHJHK, 7 DCPM => 1 BFGL',
 '7 RBJHJ, 8 CHPCH, 1 SJHGH, 9 ZMRD, 2 VDVN, 17 SFSN, 4 DPZW => 9 TXWFP',
 '1 KBJXG, 6 GJSNW, 2 RKBM => 9 QMVN',
 '1 QMVN, 1 MWXF => 9 QZRM']

# Solulu

Inspired by [asmartins](https://asmartins.com/blog/rocket-fuel/)

In [3]:
reactions = dict()

for line in in_:
    req, prod = line.split(" => ")
    amount_prod, chemical_prod = prod.split()
    reactions[chemical_prod] = [int(amount_prod), dict()]
    components = reactions[chemical_prod][-1]

    for component in req.split(", "):
        amount_req, chemical_req = component.split()
        components[chemical_req] = int(amount_req)

In [4]:
ORE, FUEL = "ORE", "FUEL"


def calc_ore_required(amount_fuel):
    res = 0
    queue = defaultdict(int, {FUEL: amount_fuel})
    stock = defaultdict(int)

    while queue:
        chemical, amount = queue.popitem()
        in_stock = stock[chemical]
        out_of_stock = amount - in_stock
        stock[chemical] = 0 if out_of_stock else (in_stock - amount)
        from_stock = in_stock - stock[chemical]

        amount -= from_stock
        amount_produced, components = reactions[chemical]
        n = ceil(amount / amount_produced)
        stock[chemical] = (amount_produced * n) - amount

        for component, amount_required in components.items():
            if component == ORE:
                res += n * amount_required
            else:
                queue[component] += n * amount_required

    return res

## Part 1

In [5]:
calc_ore_required(amount_fuel=1)

202617

## Part 2

In [6]:
def bisect(func, target: int, lo: int, hi: int):
    while lo <= hi:
        mid = lo + (hi - lo) // 2
        f_mid = func(mid)
        if f_mid == target:
            return mid
        if f_mid < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return lo - 1


def calc_hilo(amount):
    part1_res = calc_ore_required(amount_fuel=1)
    lo = amount // part1_res
    log = log10(lo)
    ore_hi = calc_ore_required(lo)

    while ore_hi < amount:
        log += 1
        ore_hi = calc_ore_required(amount_fuel=10**log)

    ratio = ore_hi // 10**log
    hi = int(amount // ratio)
    return lo, hi

In [7]:
TRILLION = int(1e12)

lo, hi = calc_hilo(TRILLION)
bisect(func=calc_ore_required, target=TRILLION, lo=lo, hi=hi)

7863863